In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [35]:
df = pd.read_csv("sg_companies_reviews.csv")

In [29]:
df.isna().sum()

Company Name         0
Overall Rating       0
Review Date          0
Review Title        66
Job Title            5
Job Details          0
Location          2763
Pros                 0
Cons                 0
dtype: int64

In [19]:
def format_date(x, input_format = "%d-%b-%y", ignore_errors = False):
    try:
        return datetime.strptime(x, input_format)
    except Exception as e:
        if ignore_errors:
            return x
        else:
            raise e

def clean_df_dates(df, date_col, **args):
    df[date_col] = df[date_col].apply(lambda x: format_date(x, **args))
    return df

df = clean_df_dates(df, "Review Date", ignore_errors = True)

In [38]:
from cleantext import clean

def is_ascii(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
        return True
    except UnicodeDecodeError:
        return False

def get_non_ascii_strings(df, col):
    # Use apply function to check each value in the specified column
    non_ascii_mask = df[col].apply(lambda x: not is_ascii(str(x)))

    # Return a DataFrame containing non-ASCII strings
    return df[non_ascii_mask]

def clean_text(df, col):
    df[col] = df[col].apply(lambda x: clean(x))
    return df

column_to_check = 'Review Title'
df = clean_text(df, column_to_check)
result = get_non_ascii_strings(df, column_to_check)
print(f"Non-ASCII strings in '{column_to_check}':")
print(len(result))
display(result)


Non-ASCII strings in 'Review Title':
1


,Company Name,Overall Rating,Review Date,Review Title,Job Title,Job Details,Location,Pros,Cons
4088,Genpact,1,45331,💩jaipur sitapura uper management,Process Developer Operations,"Current Employee, less than 1 year",NaN,"Never ever come to this location of Genpact, j...",Employees hard work never seen by management a...
